In [27]:
#Importing modules
import pandas as pd
import numpy as np
import sklearn
merged_data = pd.read_csv("C:/Users/Mr.Hassan/Desktop/100DAYSOFML/BCG/Churn_Model/clean_data_after_eda.csv")
price_df = pd.read_csv(r"C:\Users\Mr.Hassan\Desktop\100DAYSOFML\BCG\Churn_Eda\price_data.csv")

---

# Feature engineering

### 1. Difference between off-peak prices in December and preceding January

This first feature is called Difference between off-peak prices in December and preceding January.I will build on this to create more features with predictive power.

In [28]:
#Converting reference date("Price_date") to datetime with pandas
price_df["price_date"] = pd.to_datetime(price_df["price_date"],format="%Y-%m-%d")

In [29]:
#Findng the average prices for the off peak prices
monthly_price_by_date = price_df.groupby(["id","price_date"]).agg({"price_off_peak_var": "mean", "price_off_peak_fix": "mean"}).reset_index()
monthly_price_by_date

,id,price_date,price_off_peak_var,price_off_peak_fix
0,0002203ffbb812588b632b9e628cc38d,2015-01-01,0.126098,40.565969
1,0002203ffbb812588b632b9e628cc38d,2015-02-01,0.126098,40.565969
2,0002203ffbb812588b632b9e628cc38d,2015-03-01,0.128067,40.728885
3,0002203ffbb812588b632b9e628cc38d,2015-04-01,0.128067,40.728885
4,0002203ffbb812588b632b9e628cc38d,2015-05-01,0.128067,40.728885
...,...,...,...,...
192997,ffff7fa066f1fb305ae285bb03bf325a,2015-08-01,0.119916,40.728885
192998,ffff7fa066f1fb305ae285bb03bf325a,2015-09-01,0.119916,40.728885
192999,ffff7fa066f1fb305ae285bb03bf325a,2015-10-01,0.119916,40.728885
193000,ffff7fa066f1fb305ae285bb03bf325a,2015-11-01,0.119916,40.728885


In [30]:
jan_prices = monthly_price_by_date.groupby("id").first().reset_index()
dec_prices = monthly_price_by_date.groupby("id").last().reset_index()

#Difference between off peak prices for energy
offpeak_diff_var = dec_prices["price_off_peak_var"] - jan_prices["price_off_peak_var"]
#Difference between off peak prices for gas
offpeak_diff_fix = dec_prices["price_off_peak_fix"] - jan_prices["price_off_peak_fix"]

diff = pd.concat([offpeak_diff_var,offpeak_diff_fix],axis =1)
diff = diff.rename(columns={'price_off_peak_var': 'dec_1', 'price_off_peak_fix': 'dec_2'})
df = pd.concat([merged_data,diff],axis =1)
df

,Unnamed: 0,id,mean_price_off_peak_var,mean_price_peak_var,mean_price_mid_peak_var,mean_price_off_peak_fix,mean_price_peak_fix,mean_price_mid_peak_fix,mean_year_price_p1,mean_year_price_p2,...,imp_cons,margin_gross_pow_ele,margin_net_pow_ele,nb_prod_act,net_margin,num_years_antig,origin_up,pow_max,dec_1,dec_2
0,0.0,0002203ffbb812588b632b9e628cc38d,0.124338,0.103794,0.073160,40.701732,24.421038,16.280694,24.524832,40.826071,...,40.78,43.08,43.08,1.0,81.42,6.0,kamkkxfxxuwbdslkwifmmcsiusiuosws,17.250,-0.006192,0.162916
1,1.0,0004351ebdd665e6ee664792efc4fd13,0.146426,0.000000,0.000000,44.385450,0.000000,0.000000,0.000000,44.531877,...,0.00,24.42,24.42,1.0,61.58,6.0,kamkkxfxxuwbdslkwifmmcsiusiuosws,13.200,-0.004104,0.177779
2,2.0,0010bcc39e42b3c2131ed2ce55246e3c,0.181558,0.000000,0.000000,45.319710,0.000000,0.000000,0.000000,45.501268,...,213.76,38.58,38.58,2.0,81.61,3.0,lxidpiddsbxsbosboudacockeimpuepw,13.856,0.050443,1.500000
3,3.0,00114d74e963e47177db89bc70108537,0.147926,0.000000,0.000000,44.266930,0.000000,0.000000,0.000000,44.414856,...,0.00,29.76,29.76,1.0,157.99,6.0,kamkkxfxxuwbdslkwifmmcsiusiuosws,13.200,-0.010018,0.162916
4,4.0,0013f326a839a2f6ad87a1859952d227,0.126076,0.105542,0.074921,40.728885,24.437330,16.291555,24.542872,40.854961,...,195.20,30.00,30.00,1.0,341.58,3.0,lxidpiddsbxsbosboudacockeimpuepw,20.000,-0.003994,-0.000001
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16091,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.050232,-0.335085
16092,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.003778,0.177779
16093,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.001760,0.164916
16094,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.009391,0.162916


---
## 2. Average Price Changes Across Periods
This second feature is the average price changes across the three periods

In [31]:
#Average Price Changes for Energy(VAR)
df["off_peak_peak_var_mean_diff"] = df["mean_price_off_peak_var"]- df["mean_price_peak_var"]
df["off_peak_mid_peak_var_mean_diff"] = df["mean_price_off_peak_var"]-df["mean_price_mid_peak_var"]
df["peak_mid_peak_var_mean_diff"] = df["mean_price_peak_var"]-df["mean_price_mid_peak_var"]

#Average Price Changes for Gas (VAR)
df["off_peak_peak_fix_mean_diff"] = df["mean_price_off_peak_fix"]-df["mean_price_peak_fix"]
df["off_peak_mid_peak_fix_mean_diff"] = df["mean_price_off_peak_fix"]-df["mean_price_mid_peak_fix"]
df["peak_mid_peak_fix_mean_diff"] = df["mean_price_peak_fix"]-df["mean_price_mid_peak_fix"]
df

,Unnamed: 0,id,mean_price_off_peak_var,mean_price_peak_var,mean_price_mid_peak_var,mean_price_off_peak_fix,mean_price_peak_fix,mean_price_mid_peak_fix,mean_year_price_p1,mean_year_price_p2,...,origin_up,pow_max,dec_1,dec_2,off_peak_peak_var_mean_diff,off_peak_mid_peak_var_mean_diff,peak_mid_peak_var_mean_diff,off_peak_peak_fix_mean_diff,off_peak_mid_peak_fix_mean_diff,peak_mid_peak_fix_mean_diff
0,0.0,0002203ffbb812588b632b9e628cc38d,0.124338,0.103794,0.073160,40.701732,24.421038,16.280694,24.524832,40.826071,...,kamkkxfxxuwbdslkwifmmcsiusiuosws,17.250,-0.006192,0.162916,0.020545,0.051178,0.030633,16.280694,24.421038,8.140345
1,1.0,0004351ebdd665e6ee664792efc4fd13,0.146426,0.000000,0.000000,44.385450,0.000000,0.000000,0.000000,44.531877,...,kamkkxfxxuwbdslkwifmmcsiusiuosws,13.200,-0.004104,0.177779,0.146426,0.146426,0.000000,44.385450,44.385450,0.000000
2,2.0,0010bcc39e42b3c2131ed2ce55246e3c,0.181558,0.000000,0.000000,45.319710,0.000000,0.000000,0.000000,45.501268,...,lxidpiddsbxsbosboudacockeimpuepw,13.856,0.050443,1.500000,0.181558,0.181558,0.000000,45.319710,45.319710,0.000000
3,3.0,00114d74e963e47177db89bc70108537,0.147926,0.000000,0.000000,44.266930,0.000000,0.000000,0.000000,44.414856,...,kamkkxfxxuwbdslkwifmmcsiusiuosws,13.200,-0.010018,0.162916,0.147926,0.147926,0.000000,44.266930,44.266930,0.000000
4,4.0,0013f326a839a2f6ad87a1859952d227,0.126076,0.105542,0.074921,40.728885,24.437330,16.291555,24.542872,40.854961,...,lxidpiddsbxsbosboudacockeimpuepw,20.000,-0.003994,-0.000001,0.020534,0.051155,0.030621,16.291555,24.437330,8.145775
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16091,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,-0.050232,-0.335085,NaN,NaN,NaN,NaN,NaN,NaN
16092,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,-0.003778,0.177779,NaN,NaN,NaN,NaN,NaN,NaN
16093,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,-0.001760,0.164916,NaN,NaN,NaN,NaN,NaN,NaN
16094,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,-0.009391,0.162916,NaN,NaN,NaN,NaN,NaN,NaN


## 3. Maximum Price Changes Across Periods
This third feature is the maximum price changes across the three periods

In [32]:
max_diff_across_periods_months = df.groupby("id").agg({
    'off_peak_peak_var_mean_diff': 'max',
    'peak_mid_peak_var_mean_diff': 'max',
    'off_peak_mid_peak_var_mean_diff': 'max',
    'off_peak_peak_fix_mean_diff': 'max',
    'peak_mid_peak_fix_mean_diff': 'max',
    'off_peak_mid_peak_fix_mean_diff': 'max'
}).reset_index().rename(
    columns={
        'off_peak_peak_var_mean_diff': 'off_peak_peak_var_max_monthly_diff',
        'peak_mid_peak_var_mean_diff': 'peak_mid_peak_var_max_monthly_diff',
        'off_peak_mid_peak_var_mean_diff': 'off_peak_mid_peak_var_max_monthly_diff',
        'off_peak_peak_fix_mean_diff': 'off_peak_peak_fix_max_monthly_diff',
        'peak_mid_peak_fix_mean_diff': 'peak_mid_peak_fix_max_monthly_diff',
        'off_peak_mid_peak_fix_mean_diff': 'off_peak_mid_peak_fix_max_monthly_diff'
    }
)
df = pd.merge(df,max_diff_across_periods_months,on="id")
df

,Unnamed: 0,id,mean_price_off_peak_var,mean_price_peak_var,mean_price_mid_peak_var,mean_price_off_peak_fix,mean_price_peak_fix,mean_price_mid_peak_fix,mean_year_price_p1,mean_year_price_p2,...,peak_mid_peak_var_mean_diff,off_peak_peak_fix_mean_diff,off_peak_mid_peak_fix_mean_diff,peak_mid_peak_fix_mean_diff,off_peak_peak_var_max_monthly_diff,peak_mid_peak_var_max_monthly_diff,off_peak_mid_peak_var_max_monthly_diff,off_peak_peak_fix_max_monthly_diff,peak_mid_peak_fix_max_monthly_diff,off_peak_mid_peak_fix_max_monthly_diff
0,0.0,0002203ffbb812588b632b9e628cc38d,0.124338,0.103794,0.073160,40.701732,24.421038,16.280694,24.524832,40.826071,...,0.030633,16.280694,24.421038,8.140345,0.020545,0.030633,0.051178,16.280694,8.140345,24.421038
1,1.0,0004351ebdd665e6ee664792efc4fd13,0.146426,0.000000,0.000000,44.385450,0.000000,0.000000,0.000000,44.531877,...,0.000000,44.385450,44.385450,0.000000,0.146426,0.000000,0.146426,44.385450,0.000000,44.385450
2,2.0,0010bcc39e42b3c2131ed2ce55246e3c,0.181558,0.000000,0.000000,45.319710,0.000000,0.000000,0.000000,45.501268,...,0.000000,45.319710,45.319710,0.000000,0.181558,0.000000,0.181558,45.319710,0.000000,45.319710
3,3.0,00114d74e963e47177db89bc70108537,0.147926,0.000000,0.000000,44.266930,0.000000,0.000000,0.000000,44.414856,...,0.000000,44.266930,44.266930,0.000000,0.147926,0.000000,0.147926,44.266930,0.000000,44.266930
4,4.0,0013f326a839a2f6ad87a1859952d227,0.126076,0.105542,0.074921,40.728885,24.437330,16.291555,24.542872,40.854961,...,0.030621,16.291555,24.437330,8.145775,0.020534,0.030621,0.051155,16.291555,8.145775,24.437330
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14594,14594.0,ffebf6a979dd0b17a41076df1057e733,0.122426,0.102375,0.072210,40.606699,24.364017,16.242678,24.466392,40.729125,...,0.030165,16.242682,24.364021,8.121339,0.020050,0.030165,0.050215,16.242682,8.121339,24.364021
14595,14595.0,fffac626da707b1b5ab11e8431a4d0a2,0.147137,0.000000,0.000000,44.311375,0.000000,0.000000,0.000000,44.458512,...,0.000000,44.311375,44.311375,0.000000,0.147137,0.000000,0.147137,44.311375,0.000000,44.311375
14596,14596.0,fffc0cacd305dd51f316424bbb08d1bd,0.153879,0.129497,0.094842,41.160171,24.895768,16.763569,25.025265,41.314049,...,0.034655,16.264402,24.396601,8.132199,0.024382,0.034655,0.059037,16.264402,8.132199,24.396601
14597,14597.0,fffe4f5646aa39c7f97f95ae2679ce64,0.123858,0.103499,0.073735,40.606699,24.364017,16.242678,24.467516,40.730558,...,0.029764,16.242682,24.364021,8.121339,0.020359,0.029764,0.050123,16.242682,8.121339,24.364021
